In [8]:
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import tensorflow as tf

from functools import partial

2025-05-15 21:55:45.702765: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-15 21:55:45.848373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747364145.901754    4364 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747364145.916000    4364 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-15 21:55:46.055845: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Plan
- Build Facial Detection model first
- Then build Facial Recognition model

## Data

Plan for data: 
1. Get a dataset of just faces
2. For Facial Detection model, combine this dataset with the ImageNet dataset
3. For Facial Recognition model, combine face dataset with pictures of approved faces(ie. mine) 

In [2]:
DATAPATH = '/home/misha/Desktop/data/face_recognition_Data'

In [3]:
notFace_path = DATAPATH + '/Not Face'

In [4]:
### Data clean up functionality
for obj_folder in os.listdir(notFace_path):
    if not os.path.isdir(notFace_path + f'/{obj_folder}'):
        continue
    else:
        print(f"Found folder {obj_folder}")
        for image_path in os.listdir(notFace_path + f'/{obj_folder}'):
            try:
                shutil.move(notFace_path + f'/{obj_folder}/{image_path}', notFace_path)
            except Exception as err:
                print(f"Error with file: {image_path}")
                print(err)

In [20]:
df = tf.keras.utils.image_dataset_from_directory(
    DATAPATH, 
    labels='inferred',
    color_mode='rgb',
    batch_size=24, 
    label_mode='binary',   ## Vector Representation (Use categorical_crossentropy loss)
    image_size=(640, 480),
    crop_to_aspect_ratio=True,
    seed=1, 
    shuffle=True
)

Found 3937 files belonging to 2 classes.


In [21]:
df

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 640, 480, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [5]:
# buffer_size = DATASET_SIZE
# seed = 101
# df = df.shuffle(buffer_size, seed=seed)

In [4]:
# train_size = int(TRAIN_RATIO * DATASET_SIZE)
# val_size = int(VAL_RATIO * DATASET_SIZE)
# test_size = DATASET_SIZE - train_size - val_size 

In [3]:
# train_df = df.take(train_size)
# val_df = df.skip(train_size).take(val_size)
# test_df = df.skip(train_size + val_size) # Takes the rest

In [2]:
# len(val_df)

## Face Detection Model

In [18]:
import cv2

In [19]:
image_path = '/home/misha/Desktop/data/test_images/face.jpeg'

In [20]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [21]:
img = cv2.imread(image_path)

In [22]:
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

In [23]:
faces = face_cascade.detectMultiScale(gray, 1.1, 4)

In [25]:
## Drawing rectangles around the faces
for (x,y,w,h) in faces:
    cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0, 0), 2)

In [26]:
cv2.imshow('img', img)
cv2.waitKey()

100